In [1]:
# Normalise features
# Find the optimal parameters C (inverse regulalisation strengths)
# Find the best features to predict stock movement (up or down)
# Convert probability into absolute values (stock % increase/decrease)
# SVC, RandomForest, MinMax, LongShortTermMemory

%matplotlib inline
%pylab inline

# Nice Formatting within Jupyter Notebook
%matplotlib inline
from IPython.display import display # Allows multiple displays from a single code-cell

import os
import sys
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib
import matplotlib.pyplot as plt
import datetime
import math

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from collections import OrderedDict
from sklearn.metrics import mean_squared_error


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
pd_raw = pd.read_csv(os.path.join(os.getcwd(), 'datasets', 'example.csv'))

In [3]:
# Convert all values in the dataframe to numbers, and ignore the date string format
pd_processed = pd_raw.iloc[5:-2 , :].reset_index(drop=True).apply(pd.to_numeric, errors='ignore') 
display(pd_processed)

,DATE,AMZN,AMZN.1,AMZN.2,AMZN.3,AMZN.4,AMZN.5,AMZN.6,AMZN.7,AMZN.8,...,AMZN.49,AMZN.50,AMZN.51,AMZN.52,AMZN.53,AMZN.54,AMZN.55,AMZN.56,AMZN.57,AMZN.58
0,01/01/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,357687.9900,NaN
1,02/01/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,357687.9900,NaN
2,03/01/2017,753.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359500.5900,NaN
3,04/01/2017,757.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361174.8600,NaN
4,05/01/2017,780.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,372274.6500,NaN
5,06/01/2017,795.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379687.2300,NaN
6,09/01/2017,796.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380130.8400,NaN
7,10/01/2017,795.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379644.3000,NaN
8,11/01/2017,799.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381132.5400,NaN
9,12/01/2017,813.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,388106.2800,NaN


In [4]:

class Company(object):
    def __init__(self, name, pdframe):
        self.name = name
        self.share_prices = pdframe[name]
        self.converted_dates = self.preprocess_dates_raw(pdframe["DATE"])
        self.indicator_names_dict = {
            name + ".1" : "Common Shares Outstanding",
            name + ".2" : "Avg. Basic Shares Outstanding",
            name + ".3" : "Avg. Diluted Shares Outstanding",
            name + ".4" : "Revenues",
            name + ".5" : "COGS",
            name + ".6" : "SG&A",
            name + ".7" : "R&D",
            name + ".8" : "EBIT",
            name + ".9" : "EBITDA",
            name + ".10" : "Interest expense, net",
            name + ".11" : "Abnormal Gains/Losses",
            name + ".12" : "Income Taxes",
            name + ".13" : "Net Income from Discontinued Op.",
            name + ".14" : "Net Profit",
            name + ".15" : "Dividends",
            name + ".16" : "Cash and Cash Equivalents",
            name + ".17" : "Receivables",
            name + ".18" : "Current Assets",
            name + ".19" : "Net PP&E",
            name + ".20" : "Intangible Assets",
            name + ".21" : "Goodwill",
            name + ".22" : "Total Noncurrent Assets",
            name + ".23" : "Total Assets",
            name + ".24" : "Short term debt",
            name + ".25" : "Accounts Payable",
            name + ".26" : "Current Liabilities",
            name + ".27" : "Long Term Debt",
            name + ".28" : "Total Noncurrent Liabilities",
            name + ".29" : "Total Liabilities",
            name + ".30" : "Preferred Equity",
            name + ".31" : "Share Capital",
            name + ".32" : "Treasury Stock",
            name + ".33" : "Retained Earnings",
            name + ".34" : "Equity Before Minorities",
            name + ".35" : "Minorities",
            name + ".36" : "Total Equity",
            name + ".37" : "Depreciation & Amortisation",
            name + ".38" : "Change in Working Capital",
            name + ".39" : "Cash From Operating Activities",
            name + ".40" : "Net Change in PP&E & Intangibles",
            name + ".41" : "Cash From Investing Activities",
            name + ".42" : "Cash From Financing Activities",
            name + ".43" : "Net Change in Cash",
            name + ".44" : "Free Cash Flow",
            name + ".45" : "Gross Margin",
            name + ".46" : "Operating Margin",
            name + ".47" : "Net Profit Margin",
            name + ".48" : "Return on Equity",
            name + ".49" : "Return on Assets",
            name + ".50" : "Current Ratio",
            name + ".51" : "Liabilities to Equity Ratio",
            name + ".52" : "Debt to Assets Ratio",
            name + ".53" : "EV / EBITDA",
            name + ".54" : "EV / Sales",
            name + ".55" : "Book to Market",
            name + ".56" : "Operating Income / EV",
            name + ".57" : "Market Capitalisation",
            name + ".58" : "Enterprise Value"
        }
        # Note all indicator data are raw, NaN can be filtered at a later stage
        self.indicator_pdframes_dict = {
            name + ".1" : pdframe[name + ".1"],
            name + ".2" : pdframe[name + ".2"],
            name + ".3" : pdframe[name + ".3"],
            name + ".4" : pdframe[name + ".4"],
            name + ".5" : pdframe[name + ".5"],
            name + ".6" : pdframe[name + ".6"],
            name + ".7" : pdframe[name + ".7"],
            name + ".8" : pdframe[name + ".8"],
            name + ".9" : pdframe[name + ".9"],
            name + ".10" : pdframe[name + ".10"],
            name + ".11" : pdframe[name + ".11"],
            name + ".12" : pdframe[name + ".12"],
            name + ".13" : pdframe[name + ".13"],
            name + ".14" : pdframe[name + ".14"],
            name + ".15" : pdframe[name + ".15"],
            name + ".16" : pdframe[name + ".16"],
            name + ".17" : pdframe[name + ".17"],
            name + ".18" : pdframe[name + ".18"],
            name + ".19" : pdframe[name + ".19"],
            name + ".20" : pdframe[name + ".20"],
            name + ".21" : pdframe[name + ".21"],
            name + ".22" : pdframe[name + ".22"],
            name + ".23" : pdframe[name + ".23"],
            name + ".24" : pdframe[name + ".24"],
            name + ".25" : pdframe[name + ".25"],
            name + ".26" : pdframe[name + ".26"],
            name + ".27" : pdframe[name + ".27"],
            name + ".28" : pdframe[name + ".28"],
            name + ".29" : pdframe[name + ".29"],
            name + ".30" : pdframe[name + ".30"],
            name + ".31" : pdframe[name + ".31"],
            name + ".32" : pdframe[name + ".32"],
            name + ".33" : pdframe[name + ".33"],
            name + ".34" : pdframe[name + ".34"],
            name + ".35" : pdframe[name + ".35"],
            name + ".36" : pdframe[name + ".36"],
            name + ".37" : pdframe[name + ".37"],
            name + ".38" : pdframe[name + ".38"],
            name + ".39" : pdframe[name + ".39"],
            name + ".40" : pdframe[name + ".40"],
            name + ".41" : pdframe[name + ".41"],
            name + ".42" : pdframe[name + ".42"],
            name + ".43" : pdframe[name + ".43"],
            name + ".44" : pdframe[name + ".44"],
            name + ".45" : pdframe[name + ".45"],
            name + ".46" : pdframe[name + ".46"],
            name + ".47" : pdframe[name + ".47"],
            name + ".48" : pdframe[name + ".48"],
            name + ".49" : pdframe[name + ".49"],
            name + ".50" : pdframe[name + ".50"],
            name + ".51" : pdframe[name + ".51"],
            name + ".52" : pdframe[name + ".52"],
            name + ".53" : pdframe[name + ".53"],
            name + ".54" : pdframe[name + ".54"],
            name + ".55" : pdframe[name + ".55"],
            name + ".56" : pdframe[name + ".56"],
            name + ".57" : pdframe[name + ".57"],
            name + ".58" : pdframe[name + ".58"]
        }
    
    def preprocess_dates_raw(self, dates_raw):
        return pd.Series(list(map(datetime.datetime.strptime, dates_raw, len(dates_raw)*['%d/%m/%Y'])))
    
    def series_rows_filter_na(self, series, columns, reset_index): 
        if reset_index == True:
            return series.dropna().reset_index(drop=True)
        else:
            return series.dropna()

    def data_frame_rows_filter_na(self, pdframe, columns, reset_index): 
        # filter weekends share prices
        if reset_index == True:
            return pdframe.dropna(subset=[columns]).reset_index(drop=True)
        else:
            return pdframe.dropna(subset=[columns])
    
    def baseline_predict(self, start_date_string=None, end_date_string=None, plot=False):
        # Filter nan values (weekend share prices are nan since they are not trading!
        non_nan_share_prices = self.series_rows_filter_na(series=self.share_prices, columns="AMZN", reset_index=False)
        non_nan_dates = self.converted_dates[non_nan_share_prices.index]
        
        #Persistence Model Forecast, basically, the same share price with 1 date timelag
        # e.g. the predicted share price at time t, is t-1
        predictions = non_nan_share_prices.shift(1)
        
        #display(predictions)
        # Special case of the first value is changed to the zero 
        predictions.set_value(predictions.index[0], non_nan_share_prices[non_nan_share_prices.index[0]])
        #display(predictions)
 
        # When no date parameters are passed
        if start_date_string == None and end_date_string == None:
            x_axis = non_nan_dates
            plt.title(self.name + "\'s share price in available dates")

        else:
            # start_date and end_date should be in the format of "31/12/2018"
            start_date = self.convert_date_string_to_datetime(start_date_string)
            end_date = self.convert_date_string_to_datetime(end_date_string)
            x_axis = non_nan_dates[(non_nan_dates>=start_date) & (non_nan_dates<=end_date)]
            predictions = predictions[x_axis.index]
            plt.title(self.name + "\'s baseline predicted share price between " + start_date_string + " and " + end_date_string)
        
        if plot:
            formatter = matplotlib.dates.DateFormatter('%d/%m/%Y')            
            plt.plot(x_axis, predictions, '-', label="Predicted prices")
            plt.plot(x_axis, self.share_prices[predictions.index], '-', label="Actual prices")
            ax = pyplot.gcf().axes[0] 
            ax.xaxis.set_major_formatter(formatter)
            ax.legend()
            plt.gcf().autofmt_xdate(rotation=25)
            plt.gcf().set_size_inches(15, 10)
            plt.xlabel("Time")
            plt.ylabel("Share Price")
            plt.show()
        return predictions
        
        
    def get_share_prices(self, start_date_string=None, end_date_string=None):
        # Filter nan values (weekend share prices are nan since they are not trading!)
        non_nan_share_prices = self.series_rows_filter_na(series=self.share_prices, columns="AMZN", reset_index=False)
        non_nan_dates = self.converted_dates[non_nan_share_prices.index]
        
        # When no date parameters are passed
        if start_date_string == None and end_date_string == None:
            return non_nan_share_prices
        else:
            # start_date and end_date should be in the format of "31/12/2018"
            start_date = self.convert_date_string_to_datetime(start_date_string)
            end_date = self.convert_date_string_to_datetime(end_date_string)
            revelant_dates = non_nan_dates[(non_nan_dates>=start_date) & (non_nan_dates<=end_date)]
            relevant_share_prices = non_nan_share_prices[revelant_dates.index]
            return relevant_share_prices
    
    def convert_date_string_to_datetime(self, date_string):
        date_day, date_month, date_year = date_string.split("/")
        return datetime.datetime(int(date_year), int(date_month), int(date_day), 0, 0)

    def get_company_attribute_indicator_dict(self, indicator):
        attribute_indicator = dict()
        for i in range(1, 58+1):
            attribute_indicator[company_name + "." + str(i)] = indicator[str(i)]
            
    def show_pairplots(self):
        # Visualise the distribution of each attribute with share price
        selected_attributes = sorted(list(self.indicator_pdframes_dict.keys()))
        i = 5
        while i <= len(selected_attributes):
            sns.pairplot(data=pd_processed, x_vars=selected_attributes[i-5:i], y_vars = 'AMZN', size=3)
            i+=5

        #sns.pairplot(data=pd_processed, x_vars=selected_attributes[:10], y_vars = selected_attributes[:10], size=3)
    
    def get_correlation_between_attributes_and_price(self):
        # A) Calculating correlation coefficients between each attribtue and price
        # Get a list of attributes of the dataset except price remove
        
        selected_attributes = sorted(list(self.indicator_pdframes_dict.keys()))
        selected_attributes.remove("AMZN.1")
        selected_attributes.remove("AMZN.2")
        selected_attributes.remove("AMZN.3")
        
        index_of_report_earnings_release = self.series_rows_filter_na(
            self.indicator_pdframes_dict["AMZN.4"], "AMZN.4", reset_index=False).index

        price_correlation = dict() # dictionary to keep track the correlation of attributes with prices
        nan_corr_attributes = list()
        for attribute in selected_attributes:
            corr = np.corrcoef(self.indicator_pdframes_dict[attribute][index_of_report_earnings_release], 
                               self.share_prices[index_of_report_earnings_release])
            if math.isnan(corr[0][1]):
                nan_corr_attributes.append(self.indicator_names_dict[attribute])
            else:
                price_correlation[corr[0][1]] = attribute

        print("NaN Correlation attributes: ", ", ".join(nan_corr_attributes))
        # sort a dict by value and print out the highest correlating attributes
        correlation_rank = list(price_correlation.keys())

        for key in sorted(correlation_rank, reverse=True): # sort from highest to lowest
            print(str(round(key, 3)) + " " + price_correlation[key] + ": " + self.indicator_names_dict[price_correlation[key]])
            
    def score(self, predictions, start_date_string=None, end_date_string=None):
        non_nan_share_prices = self.series_rows_filter_na(series=self.share_prices, columns="AMZN", reset_index=False)
        non_nan_dates = self.converted_dates[non_nan_share_prices.index]

        # When no date parameters are passed
        if start_date_string == None and end_date_string == None:
            rmse = sqrt(mean_squared_error(non_nan_share_prices, predictions))
            return rmse
        else:
            # start_date and end_date should be in the format of "31/12/2018"
            start_date = self.convert_date_string_to_datetime(start_date_string)
            end_date = self.convert_date_string_to_datetime(end_date_string)
            relevant_dates = non_nan_dates[(non_nan_dates>=start_date) & (non_nan_dates<=end_date)]
            rmse = sqrt(mean_squared_error(non_nan_share_prices[relevant_dates.index], predictions[relevant_dates.index]))
            return rmse




In [ ]:
def get_relevant_columns_for_company(name):
    columns = list()
    columns.append("DATE")
    columns.append(name)
    for i in range(1, 58+1):
        columns.append(name + "." +str(i))
    return columns

amzn = Company("AMZN", pd_processed[get_relevant_columns_for_company("AMZN")])
start_date = "01/01/2017"
end_date = "01/09/2017"
amzn.get_share_prices(start_date_string=start_date, end_date_string=end_date)
amzn.baseline_predict(start_date_string=start_date, end_date_string=end_date)
print("RMSE: ", amzn.score(amzn.baseline_predict(start_date_string=start_date, end_date_string=end_date, plot=True), start_date, end_date))

In [6]:
#amzn.get_correlation_between_attributes_and_price()
#amzn.show_pairplots()

In [ ]:
class CompanyLSTM(Company):
    def __init__(self, name, pdframe):
        Company.__init__(self, name, pdframe)
        self.lstm_model = None
        
    def timeseries_to_supervised(self, data, lag=1):
        # source https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
        df = pd.DataFrame(data)
        columns = [df.shift(i) for i in range(1, lag+1)]
        columns.append(df)
        df = pd.concat(columns, axis=1)
        df.fillna(0, inplace=True)
        print("Timeseries to supervised")
        display(df)
        return df
    
    # create a differenced series
    def difference(self, series, interval=1):
        diff = list()
        index = series.index
        for i in range(1, len(series)):
            value = series[index[i]] - series[index[i-1]]
            diff.append(value)
        print("Difference")
        display(pd.Series(diff))
        return pd.Series(diff)
    
    # invert differenced value
    def inverse_difference(self, history, yhat, interval=1):
        return yhat + history.values[-interval]
    
    # scale train and test data to [-1, 1]
    def scale(self, train, test):
        # fit scaler
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler = scaler.fit(train)
        # transform train
        train = train.reshape(train.shape[0], train.shape[1])
        train_scaled = scaler.transform(train)
        # transform test
        test = test.reshape(test.shape[0], test.shape[1])
        test_scaled = scaler.transform(test)
        return scaler, train_scaled, test_scaled

    # inverse scaling for a forecasted value
    def invert_scale(self, scaler, X, value):
        new_row = [x for x in X] + [value]
        array = numpy.array(new_row)
        array = array.reshape(1, len(array))
        inverted = scaler.inverse_transform(array)
        return inverted[0, -1]

    # fit an LSTM network to training data
    def fit_lstm(self, train, batch_size, nb_epoch, neurons):
        X, y = train[:, 0:-1], train[:, -1]
        X = X.reshape(X.shape[0], 1, X.shape[1])
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        for i in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
            model.reset_states()
        return model

    # make a one-step forecast
    def forecast_lstm(self, model, batch_size, X):
        X = X.reshape(1, 1, len(X))
        yhat = model.predict(X, batch_size=batch_size)
        return yhat[0,0]
    
    def train_lstm_model(self, start_date_string, end_date_string):        
        # transform data to be stationary
        raw_values = self.get_share_prices(start_date_string, end_date_string)
        diff_values = self.difference(raw_values, 1)

        # transform data to be supervised learning
        supervised_pd = self.timeseries_to_supervised(diff_values, 1)
        supervised_values = supervised_pd.values
        display(supervised_values)

        # split data into train and test-sets
        train, test = supervised_values[0:-5], supervised_values[-5:]
        train_index = raw_values[0:-5].index.tolist()
        test_index = raw_values[-5:].index.tolist()
        #display(train_index)
        #display(test_index)

        # transform the scale of the data
        scaler, train_scaled, test_scaled = self.scale(train, test)
        print("SCALED DATASET")
        display(train_scaled)
        display(test_scaled)

        # fit the model
        print("Fitting the model")
        self.lstm_model = self.fit_lstm(train_scaled, 1, 3000, 4)
        # forecast the entire training dataset to build up state for forecasting
        train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
        self.lstm_model.predict(train_reshaped, batch_size=1)
        print("Finished fitting the model")
    



In [ ]:
amzn_lstm = CompanyLSTM("AMZN", pd_processed[get_relevant_columns_for_company("AMZN")])
amzn_lstm.train_lstm_model(start_date, end_date)

In [ ]:
# walk-forward validation on the test data
predictions = pd.Series()
for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    print("X: ", X, "y: ", y)
    yhat = forecast_lstm(lstm_model, 1, X)
    # test 
    # yhat = y
    print("Prediction: ", yhat)
    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    print("Inverted scale:", yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    predictions.set_value(test_index[i], yhat)
    expected = y
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# report performance
rmse = sqrt(mean_squared_error(raw_values[-5:], predictions))

print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted

display(raw_values)
formatter = matplotlib.dates.DateFormatter('%d/%m/%Y')            
plt.plot(amzn.converted_dates[test_index], predictions, ':', marker= 'x', label="Predicted prices")
plt.plot(amzn.converted_dates[train_index + test_index], raw_values, '-', marker= ".", label="Actual prices")
ax = pyplot.gcf().axes[0] 
ax.xaxis.set_major_formatter(formatter)
ax.legend()
plt.gcf().autofmt_xdate(rotation=25)
plt.gcf().set_size_inches(15, 10)
plt.xlabel("Time")
plt.ylabel("Share Price")
plt.show()
